In [1]:
import sys
sys.path.insert(0, '..')
import lib.game_utils as gu

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
all_games_df = pd.read_csv('../../data/games/all_games_with_data.csv')

In [11]:
all_games_df.head()


,date,year,week,team,team_score,opponent,opponent_score,win,home,overtime,...,opponent_fumble_count,opponent_fumble_lost,opponent_penalty_count,opponent_penalty_yards,opponent_third_down_count,opponent_third_down_conversions,opponent_third_down_ratio,opponent_fourth_down_count,opponent_fourth_down_conversions,opponent_fourth_down_ratio
0,2010-09-09,2010,1,New Orleans Saints,14,Minnesota Vikings,9,1,1,0,...,1,0,6,60,13,5,0.38,0,0,0.0
1,2010-09-09,2010,1,Minnesota Vikings,9,New Orleans Saints,14,0,0,0,...,1,0,3,20,11,3,0.27,0,0,0.0
2,2010-09-12,2010,1,Chicago Bears,19,Detroit Lions,14,1,1,0,...,3,2,7,40,14,4,0.29,1,0,0.0
3,2010-09-12,2010,1,Detroit Lions,14,Chicago Bears,19,0,0,0,...,4,3,9,100,14,6,0.43,1,0,0.0
4,2010-09-12,2010,1,Buffalo Bills,10,Miami Dolphins,15,0,1,0,...,0,0,3,15,18,8,0.44,0,0,0.0


In [12]:
YEAR = 2020
WEEK = 5
TEAM = gu.TEAM_NAME['Saints']

df = all_games_df[(all_games_df['year'] == YEAR) &
            (all_games_df['team'] == TEAM) &
            (all_games_df['week'] < WEEK)]
df

,date,year,week,team,team_score,opponent,opponent_score,win,home,overtime,...,opponent_fumble_count,opponent_fumble_lost,opponent_penalty_count,opponent_penalty_yards,opponent_third_down_count,opponent_third_down_conversions,opponent_third_down_ratio,opponent_fourth_down_count,opponent_fourth_down_conversions,opponent_fourth_down_ratio
5144,2020-09-13,2020,1,New Orleans Saints,34,Tampa Bay Buccaneers,23,1,1,0,...,2,1,9,103,13,5,0.38,1,1,1.0
5183,2020-09-21,2020,2,New Orleans Saints,24,Las Vegas Raiders,34,0,0,0,...,2,1,3,13,17,10,0.59,2,2,1.0
5212,2020-09-27,2020,3,New Orleans Saints,30,Green Bay Packers,37,0,1,0,...,0,0,2,10,11,5,0.45,2,1,0.5
5231,2020-10-04,2020,4,New Orleans Saints,35,Detroit Lions,29,1,0,0,...,0,0,5,49,14,7,0.50,3,3,1.0


In [14]:
season_df = all_games_df[(all_games_df['year'] == YEAR)
                  & (all_games_df['team'] == TEAM)]
season_df[season_df['overtime'] == 1]

,date,year,week,team,team_score,opponent,opponent_score,win,home,overtime,...,opponent_fumble_count,opponent_fumble_lost,opponent_penalty_count,opponent_penalty_yards,opponent_third_down_count,opponent_third_down_conversions,opponent_third_down_ratio,opponent_fourth_down_count,opponent_fourth_down_conversions,opponent_fourth_down_ratio
5270,2020-10-12,2020,5,New Orleans Saints,30,Los Angeles Chargers,27,1,1,1,...,1,0,1,10,17,8,0.47,1,0,0.00
5351,2020-11-01,2020,8,New Orleans Saints,26,Chicago Bears,23,1,0,1,...,0,0,7,53,15,5,0.33,3,2,0.67
